In [ ]:

import urllib
import zipfile

import tensorflow as tf
from tensorflow.keras.optimizers import Adam

def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/satellitehurricaneimages.zip'
    urllib.request.urlretrieve(url, 'satellitehurricaneimages.zip')
    with zipfile.ZipFile('satellitehurricaneimages.zip', 'r') as zip_ref:
        zip_ref.extractall()

def preprocess(image, label):
    image = image / .255
    return image, label

def solution_model():
    download_and_extract_data()

    IMG_SIZE = 128
    BATCH_SIZE = 64
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='train/',
        image_size=(IMG_SIZE,IMG_SIZE),
        batch_size=BATCH_SIZE)

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='validation/',
        image_size=(IMG_SIZE,IMG_SIZE),
        batch_size=BATCH_SIZE)

    train_ds = train_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(
        tf.data.experimental.AUTOTUNE)
    val_ds = val_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Code to define the model
    model = tf.keras.models.Sequential([
        # Note the input shape is the desired size of the image 300x300 with 3 bytes color
        # This is the first convolution
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(0.2),
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.2),
        # The third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.2),
        # The fourth convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.2),
        # The fifth convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.2),
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Code to compile and train the model
    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])

    history = model.fit(
          train_ds,
          epochs=30,
          verbose=1,
          validation_data = val_ds)

    return model, history



Found 10000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Epoch 1/30
157/157 [==============================] - 84s 524ms/step - loss: 2.1857 - accuracy: 0.5038 - val_loss: 0.6933 - val_accuracy: 0.4985
Epoch 2/30
157/157 [==============================] - 82s 522ms/step - loss: 0.6730 - accuracy: 0.6194 - val_loss: 0.6725 - val_accuracy: 0.5580
Epoch 3/30
157/157 [==============================] - 82s 523ms/step - loss: 0.4172 - accuracy: 0.8299 - val_loss: 0.3867 - val_accuracy: 0.8180
Epoch 4/30
157/157 [==============================] - 81s 517ms/step - loss: 0.3189 - accuracy: 0.8667 - val_loss: 0.4582 - val_accuracy: 0.7810
Epoch 5/30
157/157 [==============================] - 81s 518ms/step - loss: 0.2908 - accuracy: 0.8827 - val_loss: 0.2009 - val_accuracy: 0.9275
Epoch 6/30
157/157 [==============================] - 82s 522ms/step - loss: 0.2672 - accuracy: 0.8888 - val_loss: 0.1837 - val_accuracy: 0.9275
Epoch 7/30
157/157 [===========================

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

